In [1]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain_openai import AzureOpenAI 
import pandas as pd
#import dice_ml
from pathlib import Path
from utils import *
from prompt_gen import get_prompts
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import optimizers, Sequential
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Dropout, Input
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.models import Model, load_model
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, auc, roc_curve
import os
from tqdm import tqdm
import os

2025-03-16 14:15:54.385411: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-16 14:15:54.388933: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-16 14:15:54.399683: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742148954.417673 1632703 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742148954.422850 1632703 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-16 14:15:54.442837: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:


LABELS = ["Normal", "Anomaly"]


### 🟢 Step 4: Main Execution ###

path = "data/data.csv"
df = read_data(path)
train_data, test_data = df.iloc[:276388], df.iloc[276388:]
lookback = 3
X_train, X_test, y_test, scaler = get_train_test_data(train_data, test_data, lookback)
n_features = X_train.shape[2]

# Load or train model
model_path = "lstm_autoencoder.h5"
lstm_model = load_model(model_path, custom_objects={'MeanSquaredError': tf.keras.losses.MeanSquaredError()}) if os.path.exists(model_path) else None

mse = np.mean(np.power(X_test - lstm_model.predict(X_test), 2), axis=(1, 2))
threshold = 0.04
anomaly_idx = np.where(mse > threshold)[0][0]
anomalous_sequence = X_test[anomaly_idx].reshape(1, lookback, X_train.shape[2])

feature_names = train_data.drop(columns=['class', 'type'], errors='ignore').columns.tolist()
immutable_features=["user", "role", "O", "C", "E", "A", "N"]
# Set threshold for counterfactual generation to 0.02
counterfactual_examples = generate_diverse_counterfactuals(
    lstm_model,
    anomalous_sequence,
    scaler,
    feature_names,
    threshold=0.03,
    immutable_features=immutable_features
)
evaluate_model_on_sequences(lstm_model, anomalous_sequence, counterfactual_examples, feature_names, scaler)




W0000 00:00:1742148962.954594 1632703 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


2045/2045 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step

🚀 Generating Counterfactuals...



Generating Counterfactuals:  20%|██        | 1/5 [00:02<00:08,  2.17s/cf]

✅ CF 1 is now normal (MSE=0.02985) - stopping early


Generating Counterfactuals:  40%|████      | 2/5 [00:04<00:07,  2.52s/cf]

...

✅ CF 4 is now normal (MSE=0.02990) - stopping early


Generating Counterfactuals: 100%|██████████| 5/5 [00:16<00:00,  3.33s/cf]

✅ CF 5 is now normal (MSE=0.02975) - stopping early

✅ Counterfactual Generation Complete!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step

🔍 **Model Evaluation Results:**
🛑 Anomalous Session Reconstruction Error: 0.085482

✅ Counterfactual 1 Reconstruction Error: 0.030234
                                      Difference
neutral_sites                          12.369064
internal_recipients                     6.458809
documents_copy_own_pc                   5.734789
external_recipients                     4.434732
documents_copy_own_pc_off_hour          3.984578
int_to_int_mails                        3.292809
neutral_sites_off_hour                  2.978429
out_to_out_mails                        2.674841
mails_with_attachments                  1.380025
job_search                              1.319440
logon_on_own_pc_normal                  0.864271
distinct_bcc                            0.826717
device_connects_on_own_pc_off_hour    

[{'counterfactual_reconstruction_error': 0.030233892,
  'feature_differences':                                       Difference
  neutral_sites                          12.369064
  internal_recipients                     6.458809
  documents_copy_own_pc                   5.734789
  external_recipients                     4.434732
  documents_copy_own_pc_off_hour          3.984578
  int_to_int_mails                        3.292809
  neutral_sites_off_hour                  2.978429
  out_to_out_mails                        2.674841
  mails_with_attachments                  1.380025
  job_search                              1.319440
  logon_on_own_pc_normal                  0.864271
  distinct_bcc                            0.826717
  device_connects_on_own_pc_off_hour      0.700924
  team                                    0.697029
  day_of_a_week                           0.657302
  total_emails                            0.629372
  logon_hour                              0.566246
  log

In [3]:
# 🟢 NEW: Format the anomaly + counterfactuals for LLM input
formatted_data = format_data_for_llm(anomalous_sequence, counterfactual_examples, feature_names, scaler)

# 🟢 Print or save the JSON data
print("\n🔹 **Formatted Data for LLM:**")
print(formatted_data)  # Print to console



🔹 **Formatted Data for LLM:**
{
    "anomalous_session": [
        {
            "user": 675.0,
            "logon_on_own_pc_normal": 0.0,
            "logon_on_other_pc_normal": 0.0,
            "logon_on_own_pc_off_hour": 1.0,
            "logon_on_other_pc_off_hour": 0.0,
            "logon_hour": 7.0,
            "day_of_a_week": 1.0,
            "device_connects_on_own_pc": 7.0,
            "device_connects_on_other_pc": 0.0,
            "device_connects_on_own_pc_off_hour": 0.0,
            "device_connects_on_other_pc_off_hour": 0.0,
            "documents_copy_own_pc": 22.0,
            "documents_copy_other_pc": 0.0,
            "exe_files_copy_own_pc": 0.0,
            "exe_files_copy_other_pc": 0.0,
            "documents_copy_own_pc_off_hour": 0.0,
            "documents_copy_other_pc_off_hour": 0.0,
            "exe_files_copy_own_pc_off_hour": 0.0,
            "exe_files_copy_other_pc_off_hour": 0.0,
            "neutral_sites": 89.0,
            "job_search": 0.0,
     

In [4]:

co_original = scaler.inverse_transform(anomalous_sequence.reshape(-1, len(feature_names)))
co_original_df = pd.DataFrame(co_original.reshape(3,-1),columns=counterfactual_examples[0].columns)
print(co_original_df.to_string())

    user  logon_on_own_pc_normal  logon_on_other_pc_normal  logon_on_own_pc_off_hour  logon_on_other_pc_off_hour  logon_hour  day_of_a_week  device_connects_on_own_pc  device_connects_on_other_pc  device_connects_on_own_pc_off_hour  device_connects_on_other_pc_off_hour  documents_copy_own_pc  documents_copy_other_pc  exe_files_copy_own_pc  exe_files_copy_other_pc  documents_copy_own_pc_off_hour  documents_copy_other_pc_off_hour  exe_files_copy_own_pc_off_hour  exe_files_copy_other_pc_off_hour  neutral_sites  job_search  hacking_sites  neutral_sites_off_hour  job_search_off_hour  hacking_sites_off_hour  total_emails  int_to_int_mails  int_to_out_mails  out_to_int_mails  out_to_out_mails  internal_recipients  external_recipients  distinct_bcc  mails_with_attachments  after_hour_mails  role  business_unit  functional_unit  department  team     O     C     E     A     N
0  675.0                     0.0                       0.0                       1.0                         0.0         

In [5]:
#cf_example = pd.DataFrame(counterfactual_examples[0].iloc[0])
cf_example = counterfactual_examples
#cf_example1  = cf_example.T.round(1)
print(type(cf_example))
cf_example[4]   # list of CF dataframes


<class 'list'>


,user,logon_on_own_pc_normal,logon_on_other_pc_normal,logon_on_own_pc_off_hour,logon_on_other_pc_off_hour,logon_hour,day_of_a_week,device_connects_on_own_pc,device_connects_on_other_pc,device_connects_on_own_pc_off_hour,...,role,business_unit,functional_unit,department,team,O,C,E,A,N
0,675.0,0.713983,0.007962,0.535136,0.0,7.441973,1.977825,7.373900,0.015285,0.294130,...,14.0,1.0,2.837465,2.099065,3.843745,39.0,40.0,47.0,50.0,26.0
1,675.0,0.854455,0.014584,0.401808,0.0,7.579709,2.011969,7.277054,0.001431,0.266381,...,14.0,1.0,2.961633,2.349903,3.085686,39.0,40.0,47.0,50.0,26.0
2,675.0,0.837595,0.000000,0.403395,0.0,7.557063,1.978618,7.033497,0.010309,0.221397,...,14.0,1.0,2.922812,2.143360,2.789212,39.0,40.0,47.0,50.0,26.0


In [6]:
 # Initialize Azure OpenAI client
llm = AzureChatOpenAI(
            deployment_name="gpt-4o",
            openai_api_base=os.getenv("ENDPOINT_URL", "https://g4266-m3cff6ws-swedencentral.openai.azure.com/"),
            temperature=0,
            max_tokens=4096,
            openai_api_key=os.getenv("AZURE_OPENAI_API_KEY", "6xxmkjbfwnm3Li1keZmIG0V5cggGUXPqfSdAppx2EkW0iUSXSbrDJQQJ99AKACfhMk5XJ3w3AAAAACOGdLLq"),
            openai_api_version="2024-05-01-preview"
        )


/tmp/ipykernel_1632703/1979599308.py:2: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(
/home/sathish/UEBA/.venv/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://g4266-m3cff6ws-swedencentral.openai.azure.com/ to https://g4266-m3cff6ws-swedencentral.openai.azure.com/openai.
  warnings.warn(
/home/sathish/UEBA/.venv/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_

In [7]:
system_prompt_text, human_prompt_text = get_prompts("LSTM","generate_rule")
print(system_prompt_text)
print(human_prompt_text)

You are a network security expert. Your task is to analyze anomalies detected by an ML system that flags insider threats on a network. Interpret the provided anomaly data and counterfactual cases and extract the most important observed rules.
The Immutable features, negative outcome, counterfactual cases, and inferred rules are provided below.

Immutable Features:
{immutable_features}

Negative Outcome (Anomaly) Data:
{negative_outcome}

Counterfactual Cases Data:
{positive_outcome}

Extract the most important observed rules from the provided insider anomaly data. Please explain which features are driving the anomaly and how the counterfactual cases inform the rule extraction. The Anomaly and counterfactual data is structured as sequences of 3 timesteps (lookback of 3) for each user. Each feature has values for t-2, t-1, and t. The rules should be applied to the most recent timestep (t) for each feature. ----- Rules ----- <List of Rules>


In [7]:
# Define your system prompt (this is the prompt that sets the overall behavior/context)
system_prompt_text = (
    "You are a network security expert. Your task is to analyze anomalies "
    "detected by an ML system that flags insider threats on a network. "
    "Interpret the provided anomaly data and counterfactual cases and extract "
    "the most important observed rules."
)

# Define your human prompt (where you pass the detailed instructions and data)
human_prompt_text = (
    "The Immutable features, negative outcome, counterfactual cases, and inferred rules are provided below.\n\n"
    "Immutable Features:\n{immutable_features}\n\n"
    "Negative Outcome (Anomaly) Data:\n{negative_outcome}\n\n"
    "Counterfactual Cases Data:\n{positive_outcome}\n\n"
    " extract the most important observed rules over the provided Insider Anomal Please explain which features are driving the anomaly and how the counterfactual. Maximum of 5 rules. "
    "The Anomaly and counterfactual data is structured as sequences of 3 timesteps (lookback of 3) for each user. Each feature has values for t-2, t-1, and t. The rules should be applied to the most recent timestep (t) for each feature."
    " ----- Rules -----"
       " <List of Rules>"
 
)


In [8]:
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt_text),
    HumanMessagePromptTemplate.from_template(human_prompt_text)
])

In [9]:

negative_outcome_str = prep_data(co_original_df.to_string(index=False) )
positive_outcome_str = prep_data("\n\n".join([df.to_string(index=False) for df in cf_example]))
dataset_info = get_network_activity_info()
negative_outcome = negative_outcome_str
counterfactuals = positive_outcome_str


Header:
['user', 'logon_on_own_pc_normal', 'logon_on_other_pc_normal', 'logon_on_own_pc_off_hour', 'logon_on_other_pc_off_hour', 'logon_hour', 'day_of_a_week', 'device_connects_on_own_pc', 'device_connects_on_other_pc', 'device_connects_on_own_pc_off_hour', 'device_connects_on_other_pc_off_hour', 'documents_copy_own_pc', 'documents_copy_other_pc', 'exe_files_copy_own_pc', 'exe_files_copy_other_pc', 'documents_copy_own_pc_off_hour', 'documents_copy_other_pc_off_hour', 'exe_files_copy_own_pc_off_hour', 'exe_files_copy_other_pc_off_hour', 'neutral_sites', 'job_search', 'hacking_sites', 'neutral_sites_off_hour', 'job_search_off_hour', 'hacking_sites_off_hour', 'total_emails', 'int_to_int_mails', 'int_to_out_mails', 'out_to_int_mails', 'out_to_out_mails', 'internal_recipients', 'external_recipients', 'distinct_bcc', 'mails_with_attachments', 'after_hour_mails', 'role', 'business_unit', 'functional_unit', 'department', 'team', 'O', 'C', 'E', 'A', 'N']

Consolidated Rows:
Row 1:
[[675.0, 675.

In [10]:
messages = chat_prompt.format_messages(
   immutable_features=immutable_features,
    negative_outcome=negative_outcome,
    positive_outcome=counterfactuals,
    
)

In [11]:
prompt_text = chat_prompt.format(
    immutable_features=immutable_features,
    negative_outcome=negative_outcome,
    positive_outcome=counterfactuals,
    
)


In [12]:
# Optionally print the prompt text that will be sent to the LLM
print("Prompt being sent to the LLM:")
print(prompt_text)

Prompt being sent to the LLM:
System: You are a network security expert. Your task is to analyze anomalies detected by an ML system that flags insider threats on a network. Interpret the provided anomaly data and counterfactual cases and extract the most important observed rules.
Human: The Immutable features, negative outcome, counterfactual cases, and inferred rules are provided below.

Immutable Features:
['user', 'role', 'O', 'C', 'E', 'A', 'N']

Negative Outcome (Anomaly) Data:
{'header': ['user', 'logon_on_own_pc_normal', 'logon_on_other_pc_normal', 'logon_on_own_pc_off_hour', 'logon_on_other_pc_off_hour', 'logon_hour', 'day_of_a_week', 'device_connects_on_own_pc', 'device_connects_on_other_pc', 'device_connects_on_own_pc_off_hour', 'device_connects_on_other_pc_off_hour', 'documents_copy_own_pc', 'documents_copy_other_pc', 'exe_files_copy_own_pc', 'exe_files_copy_other_pc', 'documents_copy_own_pc_off_hour', 'documents_copy_other_pc_off_hour', 'exe_files_copy_own_pc_off_hour', 'ex

In [13]:
response = llm(messages)
print("\nResponse from AzureChatOpenAI:")
response.pretty_print()

/tmp/ipykernel_1632703/296912339.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)



Response from AzureChatOpenAI:
================================== Ai Message ==================================

### Analysis of Insider Threat Anomaly Data and Counterfactual Cases

The provided data includes anomaly observations and counterfactual cases for insider threat detection. The anomaly data highlights behaviors that deviate from normal patterns, while the counterfactual cases represent hypothetical scenarios where the anomaly is mitigated. By comparing these datasets, we can extract rules that identify key features driving the anomaly and suggest thresholds or patterns for detection.

---

### Key Observations from the Data

1. **Immutable Features**:
   - The features `['user', 'role', 'O', 'C', 'E', 'A', 'N']` are immutable and cannot be altered. These features are not directly involved in anomaly detection but may provide context for behavioral patterns.

2. **Features Driving the Anomaly**:
   - **Off-hour activities**: Features such as `logon_on_own_pc_off_hour`, `logo

In [14]:
dataset_info = get_network_activity_info()
rules = response 

In [15]:
system_prompt_text,human_prompt_text = get_prompts("LSTM","generate_code")

In [16]:
print(human_prompt_text)

The Immutable features, negative outcome, counterfactual cases, and inferred rules are provided below.

Immutable Features:
{immutable_features}

Negative Outcome (Anomaly) Data:
{negative_outcome}

Counterfactual Cases Data:
{positive_outcome}

Derived Rules:
{rules}

The following info about the dataset is available:
{dataset_info}you should generate python code to count how many of the counterfactuals are consistent with the rule.do not skip any of the data in the code generated. The code should create a df with the counterfactuals provided and then check for each rule how many of them follow the rules. Order the rules. Finally, you should print the results. assume that the counterfactual data is going to be read from a file named 'counterfactuals.pkl'. and use applymap(lambda cell: cell[2] if isinstance(cell, list) else cell) to extract the values from the list in the cells.The counterfactual data is structured as sequences of 3 timesteps (lookback of 3) for each user. Each feature

In [ ]:
# Define your system prompt (this is the prompt that sets the overall behavior/context)
system_prompt_text = (
    "You are a network security expert. Your task is to analyze anomalies "
    "detected by an ML system that flags insider threats on a network. "

)

# Define your human prompt (where you pass the detailed instructions and data)
human_prompt_text = (
    "The Immutable features, negative outcome, counterfactual cases, and inferred rules are provided below.\n\n"
    "Immutable Features:\n{immutable_features}\n\n"
    "Negative Outcome (Anomaly) Data:\n{negative_outcome}\n\n"
    "Counterfactual Cases Data:\n{positive_outcome}\n\n"
    "Derived Rules:\n{rules}\n\n"
     "The following info about the dataset is available:\n{dataset_info}"
       
    "you should generate python code to count how many of the counterfactuals are consistent with the rule.do not skip any of the data in the code generated. The code should create a df with the counterfactuals provided and then check for each rule how many of them follow the rules. Order the rules. Finally, you should print the results. "
    "assume that the counterfactual data is going to be read from a file named 'counterfactuals.pkl'. and use applymap(lambda cell: cell[2] if isinstance(cell, list) else cell) to extract the values from the list in the cells."
    "The counterfactual data is structured as sequences of 3 timesteps (lookback of 3) for each user. Each feature has values for t-2, t-1, and t. The rules should be applied to the most recent timestep (t) for each feature."
        """        ----- Code -----
            ```
            import pandas as pd
            #complete this code
            ```
    """
 
)

In [17]:
print(human_prompt_text)

The Immutable features, negative outcome, counterfactual cases, and inferred rules are provided below.

Immutable Features:
{immutable_features}

Negative Outcome (Anomaly) Data:
{negative_outcome}

Counterfactual Cases Data:
{positive_outcome}

Derived Rules:
{rules}

The following info about the dataset is available:
{dataset_info}you should generate python code to count how many of the counterfactuals are consistent with the rule.do not skip any of the data in the code generated. The code should create a df with the counterfactuals provided and then check for each rule how many of them follow the rules. Order the rules. Finally, you should print the results. assume that the counterfactual data is going to be read from a file named 'counterfactuals.pkl'. and use applymap(lambda cell: cell[2] if isinstance(cell, list) else cell) to extract the values from the list in the cells.The counterfactual data is structured as sequences of 3 timesteps (lookback of 3) for each user. Each feature

In [18]:
chat_prompt1 = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt_text),
    HumanMessagePromptTemplate.from_template(human_prompt_text)
])

In [19]:

df = pd.DataFrame(counterfactuals["rows"], columns=counterfactuals["header"])
df.to_pickle("counterfactuals.pkl")

In [20]:
# Load the counterfactual data from the CSV file
df = pd.read_pickle("counterfactuals.pkl")
cf_str = df.to_string(index=False)

In [21]:
df_th = df.applymap(lambda cell: cell[2] if isinstance(cell, list) else cell)
df_th

,user,logon_on_own_pc_normal,logon_on_other_pc_normal,logon_on_own_pc_off_hour,logon_on_other_pc_off_hour,logon_hour,day_of_a_week,device_connects_on_own_pc,device_connects_on_other_pc,device_connects_on_own_pc_off_hour,...,role,business_unit,functional_unit,department,team,O,C,E,A,N
0,675.0,0.918692,0.035198,0.418662,0.035777,7.679090,2.070812,5.949072,0.006546,0.233116,...,14.0,1.0,1.181440,2.525600,2.743276,39.0,40.0,47.0,50.0,26.0
1,675.0,0.850021,0.019444,0.350859,0.026738,7.686089,2.001489,5.889037,0.011873,0.212401,...,14.0,1.0,2.090755,2.648537,2.821477,39.0,40.0,47.0,50.0,26.0
2,675.0,0.874619,0.012317,0.395705,0.000000,7.512805,1.997545,5.594750,0.022897,0.231019,...,14.0,1.0,2.297084,2.288283,2.526030,39.0,40.0,47.0,50.0,26.0
3,675.0,0.852693,0.085231,0.352738,0.225038,7.730969,2.057230,5.867164,0.022659,0.264410,...,14.0,1.0,0.000000,3.274090,2.886539,39.0,40.0,47.0,50.0,26.0
4,675.0,0.837595,0.000000,0.403395,0.000000,7.557063,1.978618,7.033497,0.010309,0.221397,...,14.0,1.0,2.922812,2.143360,2.789212,39.0,40.0,47.0,50.0,26.0


In [22]:
messages = chat_prompt1.format_messages(
   immutable_features=immutable_features,
    negative_outcome=negative_outcome,
    positive_outcome=cf_str,
    rules=rules,
    dataset_info=dataset_info
    
    
)

In [23]:
# Optionally print the prompt text that will be sent to the LLM
print("Prompt being sent to the LLM:")
print(messages)

Prompt being sent to the LLM:
[SystemMessage(content='You are a network security expert. Your task is to analyze anomalies detected by an ML system that flags insider threats on a network. Interpret the provided anomaly data and counterfactual cases and extract the most important observed rules.', additional_kwargs={}, response_metadata={}), HumanMessage(content='The Immutable features, negative outcome, counterfactual cases, and inferred rules are provided below.\n\nImmutable Features:\n[\'user\', \'role\', \'O\', \'C\', \'E\', \'A\', \'N\']\n\nNegative Outcome (Anomaly) Data:\n{\'header\': [\'user\', \'logon_on_own_pc_normal\', \'logon_on_other_pc_normal\', \'logon_on_own_pc_off_hour\', \'logon_on_other_pc_off_hour\', \'logon_hour\', \'day_of_a_week\', \'device_connects_on_own_pc\', \'device_connects_on_other_pc\', \'device_connects_on_own_pc_off_hour\', \'device_connects_on_other_pc_off_hour\', \'documents_copy_own_pc\', \'documents_copy_other_pc\', \'exe_files_copy_own_pc\', \'exe_

In [24]:
response = llm(messages)
print("\nResponse from AzureChatOpenAI:")
response.pretty_print()


Response from AzureChatOpenAI:
================================== Ai Message ==================================

```python
import pandas as pd

# Load the counterfactual data from the file
counterfactuals_file = 'counterfactuals.pkl'
df = pd.read_pickle(counterfactuals_file)

# Extract the most recent timestep (t) values from the list in the cells
df = df.applymap(lambda cell: cell[2] if isinstance(cell, list) else cell)

# Define the rules as functions
def rule_1(row):
    return (
        row['logon_on_own_pc_off_hour'] > 0.5 or
        row['logon_on_other_pc_off_hour'] > 0.4 or
        row['device_connects_on_own_pc_off_hour'] > 0.5 or
        row['device_connects_on_other_pc_off_hour'] > 0.4
    )

def rule_2(row):
    return (
        row['documents_copy_own_pc_off_hour'] > 0.3 or
        row['documents_copy_other_pc_off_hour'] > 0.3 or
        row['exe_files_copy_own_pc_off_hour'] > 0.5 or
        row['exe_files_copy_other_pc_off_hour'] > 0.5
    )

def rule_3(row):
    return (

In [25]:
def extract_code(text):
    start_code = text.find("```python") + len("```python\n")
    end_code = text.find("```", start_code)
    return text[start_code:end_code].strip()


In [26]:
text = response.content  # Get the response content from the AzureChatOpenAI response
code = extract_code(text)
code

'import pandas as pd\n\n# Load the counterfactual data from the file\ncounterfactuals_file = \'counterfactuals.pkl\'\ndf = pd.read_pickle(counterfactuals_file)\n\n# Extract the most recent timestep (t) values from the list in the cells\ndf = df.applymap(lambda cell: cell[2] if isinstance(cell, list) else cell)\n\n# Define the rules as functions\ndef rule_1(row):\n    return (\n        row[\'logon_on_own_pc_off_hour\'] > 0.5 or\n        row[\'logon_on_other_pc_off_hour\'] > 0.4 or\n        row[\'device_connects_on_own_pc_off_hour\'] > 0.5 or\n        row[\'device_connects_on_other_pc_off_hour\'] > 0.4\n    )\n\ndef rule_2(row):\n    return (\n        row[\'documents_copy_own_pc_off_hour\'] > 0.3 or\n        row[\'documents_copy_other_pc_off_hour\'] > 0.3 or\n        row[\'exe_files_copy_own_pc_off_hour\'] > 0.5 or\n        row[\'exe_files_copy_other_pc_off_hour\'] > 0.5\n    )\n\ndef rule_3(row):\n    return (\n        row[\'hacking_sites\'] > 0.5 or\n        row[\'hacking_sites_off_hou

In [27]:
import subprocess
import sys
def execute_code(code, timeout=10):
    with open("temp_code.py", "w") as f:
        f.write(code)
    try:
        result = subprocess.run([sys.executable, "temp_code.py"], capture_output=True, text=True, check=True, timeout=timeout)
        return result.stdout, False
    except subprocess.CalledProcessError as e:
        return e.stdout + e.stderr, True
    except subprocess.TimeoutExpired:
        return "Execution timed out.", True
   

In [28]:
results, _ = execute_code(code)

print(results)

Consistency of Counterfactuals with Rules:
Rule 2: File Copying During Off-hours: 5 counterfactuals consistent
Rule 4: Suspicious Email Patterns: 5 counterfactuals consistent
Rule 5: Behavioral Shifts in Neutral Sites and Job Search: 5 counterfactuals consistent
Rule 6: Elevated Total Emails and External Recipients: 2 counterfactuals consistent
Rule 1: Off-hour Logon and Device Connections: 0 counterfactuals consistent
Rule 3: Accessing Hacking Sites: 0 counterfactuals consistent



In [40]:

# Define your system prompt (this is the prompt that sets the overall behavior/context)
system_prompt_text = (
    "You are a network security expert. Your task is to analyze anomalies "
    "detected by an ML system that flags insider threats on a network. "

)

# Define your human prompt (where you pass the detailed instructions and data)
human_prompt_text = (
    " A Anomaly has been classified in the negative class. The data is the following:\n\n"
    "The Immutable features, negative outcome, counterfactual cases, and inferred rules are provided below.\n\n"
    "Immutable Features:\n{immutable_features}\n\n"
    "Negative Outcome (Anomaly) Data:\n{negative_outcome}\n\n"
    "Counterfactual Cases Data:\n{positive_outcome}\n\n"
    "Derived Rules:\n{rules}\n\n"
    "we have checked that the rules are followed by n counterfactuals:\n{results}\n\n"
     "The following info about the dataset is available:\n{dataset_info}"
       
    "Given this information, provide an explanation to the user in plain language so that he/she can improve their chances of changing class. It should be as clear as possible and call to action. Consider that the higher amount of counterfactuals that follow the rule, the more important that rule is. "

        "       ----- Explaination  ----- "
 
)


In [41]:
chat_prompt1 = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt_text),
    HumanMessagePromptTemplate.from_template(human_prompt_text)
])

In [42]:
human_prompt_text

' A Anomaly has been classified in the negative class. The data is the following:\n\nThe Immutable features, negative outcome, counterfactual cases, and inferred rules are provided below.\n\nImmutable Features:\n{immutable_features}\n\nNegative Outcome (Anomaly) Data:\n{negative_outcome}\n\nCounterfactual Cases Data:\n{positive_outcome}\n\nDerived Rules:\n{rules}\n\nwe have checked that the rules are followed by n counterfactuals:\n{results}\n\nThe following info about the dataset is available:\n{dataset_info}Given this information, provide an explanation to the user in plain language so that he/she can improve their chances of changing class. It should be as clear as possible and call to action. Consider that the higher amount of counterfactuals that follow the rule, the more important that rule is.        ----- Explaination  ----- '

In [44]:
messages = chat_prompt1.format_messages(
   immutable_features=immutable_features,
    negative_outcome=negative_outcome,
    positive_outcome=cf_str,
    rules=rules,
    dataset_info=dataset_info,
    results=results
    
    
)# Optionally print the prompt text that will be sent to the LLM
print("Prompt being sent to the LLM:")
print(messages)

Prompt being sent to the LLM:
[SystemMessage(content='You are a network security expert. Your task is to analyze anomalies detected by an ML system that flags insider threats on a network. ', additional_kwargs={}, response_metadata={}), HumanMessage(content=' A Anomaly has been classified in the negative class. The data is the following:\n\nThe Immutable features, negative outcome, counterfactual cases, and inferred rules are provided below.\n\nImmutable Features:\n[\'user\', \'role\', \'O\', \'C\', \'E\', \'A\', \'N\']\n\nNegative Outcome (Anomaly) Data:\n{\'header\': [\'user\', \'logon_on_own_pc_normal\', \'logon_on_other_pc_normal\', \'logon_on_own_pc_off_hour\', \'logon_on_other_pc_off_hour\', \'logon_hour\', \'day_of_a_week\', \'device_connects_on_own_pc\', \'device_connects_on_other_pc\', \'device_connects_on_own_pc_off_hour\', \'device_connects_on_other_pc_off_hour\', \'documents_copy_own_pc\', \'documents_copy_other_pc\', \'exe_files_copy_own_pc\', \'exe_files_copy_other_pc\', 

In [ ]:
response = llm(messages)
print("\nResponse from AzureChatOpenAI:")
response.pretty_print()

In [46]:
explaination = response.content

In [ ]:
# Define your system prompt (this is the prompt that sets the overall behavior/context)
system_prompt_text = (
    "You are a network security expert. Your task is to analyze anomalies "
    "detected by an LSTM ML system that flags insider threats on a network. "

)

# Define your human prompt (where you pass the detailed instructions and data)
human_prompt_text = (
    " A Anomaly has been classified in the negative class. The data is the following:\n\n"
    "The Immutable features, negative outcome, explaination, and dataset information are provided below.\n\n"
    "Immutable Features:\n{immutable_features}\n\n"
    "Negative Outcome (Anomaly) Data:\n{negative_outcome}\n\n"
    " The following explanation was given inorder to try and change the class.:\n{explaination}\n\n"
     "The following info about the dataset is available:\n{dataset_info}"
       
    "Given this information, provide an example in the format of a pandas dataframe that would be in the positive class for the user in tha Anomaly. note that the LSTM has a lookback of 3 .\n\n"
    " Complete the code below and note that it is very important to use the name 'temp_csv.csv', since later processes rely on it. "

        """
        import pandas as pd
        df = pd.DataFrame(...) #complete this line
        df.to_csv('temp_csv.csv', index = False)

        """
 
)


 

In [49]:

chat_prompt1 = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt_text),
    HumanMessagePromptTemplate.from_template(human_prompt_text)
])
messages = chat_prompt1.format_messages(
   immutable_features=immutable_features,
    negative_outcome=negative_outcome,
    explaination=explaination,
    dataset_info=dataset_info
    
    
)
# Optionally print the prompt text that will be sent to the LLM
print("Prompt being sent to the LLM:")
print(messages)
response = llm(messages)
print("\nResponse from AzureChatOpenAI:")
response.pretty_print()

Prompt being sent to the LLM:
[SystemMessage(content='You are a network security expert. Your task is to analyze anomalies detected by an ML system that flags insider threats on a network. ', additional_kwargs={}, response_metadata={}), HumanMessage(content=" A Anomaly has been classified in the negative class. The data is the following:\n\nThe Immutable features, negative outcome, explaination, and dataset information are provided below.\n\nImmutable Features:\n['user', 'role', 'O', 'C', 'E', 'A', 'N']\n\nNegative Outcome (Anomaly) Data:\n{'header': ['user', 'logon_on_own_pc_normal', 'logon_on_other_pc_normal', 'logon_on_own_pc_off_hour', 'logon_on_other_pc_off_hour', 'logon_hour', 'day_of_a_week', 'device_connects_on_own_pc', 'device_connects_on_other_pc', 'device_connects_on_own_pc_off_hour', 'device_connects_on_other_pc_off_hour', 'documents_copy_own_pc', 'documents_copy_other_pc', 'exe_files_copy_own_pc', 'exe_files_copy_other_pc', 'documents_copy_own_pc_off_hour', 'documents_copy